In [96]:
# 1.detect the language of the question
# 2.get approval of the detected language from the user
# 3.get the question from the user as input

# Imports

In [97]:
from langgraph.graph import StateGraph, END, START
from typing import TypedDict, List, Annotated
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
import operator
import psycopg2
from psycopg2 import sql
import re
from langchain_ollama import ChatOllama
from IPython.display import Image, display

# States

In [98]:
class States(TypedDict):
    messages: Annotated[List, operator.add]  # Accumulates all messages
    question: str  # Original user query
    language: str
    approve_language: str

static_language = "Tulu"
static_approve_language = "NO"

# Prompts

In [99]:
prompt_detect_language = """
Detect the language of the user's text. 
Respond with ONLY the language name in English. 
Do not include any other text, explanations, or punctuation.
The language is not {language}

Examples:
- User: "Hola cómo estás" → Spanish
- User: "Bonjour comment ça va" → French
- User: "Hello how are you" → English
- User: "Wie geht es dir" → German
- User: "你好吗" → Chinese
- User: "お元気ですか" → Japanese
- User: "Как дела" → Russian
- User: "سلام، حالت چطوره" → Persian
"""

In [100]:
prompt_approval = """
Analyze the user's response to determine if it means YES or NO.

CRITICAL RULES:
1. Respond with ONLY "YES" or "NO" in uppercase letters
2. No additional text, explanations, or punctuation
3. Consider context, tone, and common expressions
4. For ambiguous responses, choose the most probable interpretation

YES INDICATORS (respond with YES):
- Direct affirmatives: yes, yeah, yep, yup, sure, absolutely, definitely, certainly
- Agreement: okay, alright, fine, agreed, of course, by all means
- Positive confirmation: correct, right, exactly, that's right, I agree
- Enthusiastic: absolutely!, definitely!, without a doubt!, certainly!
- Implied yes: "I think so", "probably", "maybe", "I guess", "why not"

NO INDICATORS (respond with NO):
- Direct negatives: no, nope, nah, never, not at all, absolutely not
- Refusal: I can't, I won't, I don't think so, I'd rather not
- Negative confirmation: incorrect, wrong, that's not right, I disagree
- Hesitation: not really, not exactly, sort of but not really
- Avoidance: maybe later, another time, I'm not sure

AMBIGUOUS CASES:
- "I don't know" → NO (unless context suggests otherwise)
- "Maybe" → Consider context, but usually NO for clear decisions
- Sarcasm: Interpret literal meaning despite tone

Respond with only YES or NO:
"""

# Models

In [101]:
# Initialize the LLM
llm = ChatOllama(model="llama3.2:3b", reasoning=False, temperature=0.1)

# Nodes

In [102]:
def get_question(state: States) -> States:
    """Get the question from the user"""

    print("⚡️ Getting the question")

    user_input = input("How can I help you?")

    print(f"📡 Question: {user_input}")

    state["question"] = user_input

    return state

In [103]:
def detect_language(state: States) -> States:
    """Detect the langugae of the question"""

    print("⚡️ Detecting Language...")

    print(f"⚙️ {detect_language.__name__} Step 1: {state.get("language", static_language)}")
    chat = [
        SystemMessage(content=prompt_detect_language.format(
            language=state["language"]
        )),
        HumanMessage(content=state["question"])
    ]

    print(f"⚙️ {detect_language.__name__} Step 2")
    language = llm.invoke(chat).content

    print(f"📡 Language Detected:\n{language}")

    print(f"⚙️ {detect_language.__name__} Step 3")

    return {
        "messages": [],
        "language": language,
        "question": state["question"],
        "approve_language": state.get("approve_language", static_approve_language)
    }
    
        

In [104]:
def approve_language(state: States) -> States:
    """Ask user to approve detected language is correct or not"""

    print("⚡️ Approving Detected Language...")

    user_input = input("Do you confirm detected language?")

    print(f"⚙️ {approve_language.__name__} Step 1: {user_input}")

    chat = [
        SystemMessage(content=prompt_approval),
        HumanMessage(content=user_input)
    ]

    approve = llm.invoke(chat).content

    print(f"📡 Approve Language:\n{approve}")

    return {
        "messages": [],
        "question": state["question"],
        "language": state["language"],
        "approve_language": approve
    }
    
    

In [105]:
def is_language_approved(state: States) -> States:
    """Check the detected language is approved by the user"""

    approved = state['approve_language']

    print(f"💡 Is Language Approved: {approved}")

    return approved

# Workflow

In [106]:
def create_workflow():
    workflow = StateGraph(States)
    
    workflow.add_node(get_question.__name__, get_question)
    workflow.add_node(detect_language.__name__, detect_language)
    workflow.add_node(approve_language.__name__, approve_language)
    workflow.add_node("summerize", lambda state: state)


    workflow.add_edge(START, get_question.__name__)
    workflow.add_edge(get_question.__name__, detect_language.__name__)
    workflow.add_edge(detect_language.__name__, approve_language.__name__)


    workflow.add_conditional_edges(
        approve_language.__name__,
        is_language_approved,
        {
            "YES": "summerize",
            "NO": detect_language.__name__
        }
    )

    workflow.add_edge("summerize", END)


    return workflow.compile()

# Initiation

In [107]:

agent = create_workflow()

initiate_state = {
    "messages": [],
    "question": "",
    "language": static_language,
    "approve_language": static_approve_language
    
}

agent.invoke(initiate_state)

⚡️ Getting the question
📡 Question: آهنگ پر طرفدار
⚡️ Detecting Language...
⚙️ detect_language Step 1: Tulu
⚙️ detect_language Step 2
📡 Language Detected:
Persian
⚙️ detect_language Step 3
⚡️ Approving Detected Language...
⚙️ approve_language Step 1: بله
📡 Approve Language:
YES
💡 Is Language Approved: YES


{'messages': [],
 'question': 'آهنگ پر طرفدار',
 'language': 'Persian',
 'approve_language': 'YES'}